# AADT Training

## Imports

In [309]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import torch
import random
import cv2
from tqdm import tqdm
from matplotlib import pyplot as plt
import segmentation_models_pytorch as smp
import albumentations as album
from PIL import Image
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import random_split
from sklearn.metrics import accuracy_score
import plotly.express as px
import torchmetrics
from torchmetrics import MeanAbsolutePercentageError


%matplotlib inline

## Global Variables

In [310]:
IMG_SIZE = 1024
VEHICLE_DETECTION_COUNT_PATH = '/home/ah2719/FYP/Spatial_Finance_Transport/data/vehicle_counts_detection.csv'
DAILY_COUNT_PATH = '/home/ah2719/FYP/Spatial_Finance_Transport/data/ground_truth_data/df_aadt_week.csv'
SPEED_DATA_PATH = "/home/ah2719/FYP/Spatial_Finance_Transport/data/ground_truth_data/avg_mph.csv"
ROAD_WIDTH_PATH = ""
TIMESTAMP_PATH = "/home/ah2719/FYP/Spatial_Finance_Transport/data/ground_truth_data/timestamp.csv"
AVG_MPH_PATH = "/home/ah2719/FYP/Spatial_Finance_Transport/data/ground_truth_data/avg_mph.csv"

NN_MODEL_PATH = "/home/ah2719/FYP/Spatial_Finance_Transport/models/nn_model.pth"

## True Count Data
- Traffic monitoring stations for long-term traffic count data
    - Extract at same time as Satellite Image!
- How to use permanent and temporary traffic count stations

In [311]:
# Get Ground Truth Data

## Vehicle detection number
From vehicle detection model

## Road characteristics
From road characterstics pipeline

Includes:
- Road width
- Live speed data
- Directionality

## Neural Network Model

In [312]:
class CustomDataset(Dataset):
    def __init__(self):
        self.labels = torch.tensor(pd.read_csv(DAILY_COUNT_PATH)['aadt'].values.astype('float32'))
        self.vehicle_count = torch.tensor(pd.read_csv(DAILY_COUNT_PATH)['total_volume_normalised'].values.astype('float32')).unsqueeze(1) # Training
        #self.speed_data = pd.read_csv(SPEED_DATA_PATH) 
        #self.road_width = pd.read_csv(ROAD_WIDTH_PATH)
        self.hour = torch.tensor(pd.read_csv(DAILY_COUNT_PATH)['hour'].values.astype('float32')).unsqueeze(1)
        self.avg_mph = torch.tensor(pd.read_csv(DAILY_COUNT_PATH)['avg_mph'].values.astype('float32')).unsqueeze(1)
        self.day = torch.tensor(pd.read_csv(DAILY_COUNT_PATH)['day'].values.astype('float32')).unsqueeze(1)
        self.month = torch.tensor(pd.read_csv(DAILY_COUNT_PATH)['month'].values.astype('float32')).unsqueeze(1)

        self.x = torch.concat((self.vehicle_count, self.avg_mph, self.day, self.month, self.hour), dim=-1)
        self.y = self.labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

custom_data = CustomDataset()
train_split = 0.8
train_data, val_data = random_split(custom_data, [train_split, 1-train_split])

In [313]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(5, 5),
            nn.Linear(5,20),
            nn.Linear(20,1),
            nn.ReLU()
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [314]:
nn_model = NeuralNetwork()
nn_model

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=5, out_features=5, bias=True)
    (1): Linear(in_features=5, out_features=20, bias=True)
    (2): Linear(in_features=20, out_features=1, bias=True)
    (3): ReLU()
  )
)

In [315]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

nn_model.apply(init_weights)

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=5, out_features=5, bias=True)
    (1): Linear(in_features=5, out_features=20, bias=True)
    (2): Linear(in_features=20, out_features=1, bias=True)
    (3): ReLU()
  )
)

In [316]:
learning_rate = 1e-1
batch_size = 1
epochs = 1
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(nn_model.parameters(), lr=learning_rate)

MAPE = MeanAbsolutePercentageError()

### DataLoaders

In [317]:
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=False, sampler=None,
                    batch_sampler=None, num_workers=0, collate_fn=None,
                    pin_memory=False, drop_last=False, timeout=0,
                    worker_init_fn=None, prefetch_factor=2,
                    persistent_workers=False)

In [318]:
val_dataloader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=False, sampler=None,
                    batch_sampler=None, num_workers=0, collate_fn=None,
                    pin_memory=False, drop_last=False, timeout=0,
                    worker_init_fn=None, prefetch_factor=2,
                    persistent_workers=False)

### Training & Testing

In [319]:
def run_epoch(ep_id, action, loader, model, optimizer, criterion):
    losses = [] # Keep list of accuracies to track progress
    is_training = action == "train" # True when action == "train", else False 

    # Looping over all batches
    for batch_idx, batch in enumerate(loader): 
            x, y = batch
            #print("x: {}".format(x))
            #print("y: {}".format(y))

            # Resetting the optimizer gradients
            optimizer.zero_grad()

            # Setting model to train or test
            with torch.set_grad_enabled(is_training):
                
                # Feed batch to model
                logits = nn_model(x).squeeze(1)
                print("logits: {}".format(logits))

                # Calculate the loss based on predictions and real labels
                loss = criterion(logits, y)
                mape_loss = MAPE(logits, y)
                print("MAPE loss: {}".format(mape_loss))

                # If training, perform backprop and update weights
                if is_training:
                    loss.backward()
                    optimizer.step()

                # Append current batch accuracy
                losses.append(mape_loss.detach().numpy())

                # Print some stats every 50th batch 
                if batch_idx % 50 == 0:
                    print(f"{action.capitalize()}ing, Epoch: {ep_id+1}, Batch {batch_idx}: Loss = {loss.item()}")
    # Return accuracies to main loop                 
    return losses

In [320]:
def main(epochs, train_dl, val_dl, model, optimizer, criterion):

    # Keep lists of accuracies to track performance on train and test sets
    train_losses = []
    val_losses = []

    # Looping over epochs
    for epoch in range(epochs):
        
        # Looping over train set and training
        train_loss = run_epoch(epoch, "train", train_dl, model, optimizer, criterion)

        # Looping over test set
        val_loss = run_epoch(epoch, "val", val_dl, model, optimizer, criterion) 

        # Collecting stats
        train_losses += train_loss
        val_losses += val_loss         
            
    return train_losses, val_losses

In [321]:
train_losses, val_losses = main(epochs=epochs, train_dl=train_dataloader, val_dl=val_dataloader, model=nn_model, optimizer=optimizer, criterion=loss_fn)

logits: tensor([20.5116], grad_fn=<SqueezeBackward1>)
MAPE loss: 0.9988774657249451
Training, Epoch: 1, Batch 0: Loss = 333143904.0
logits: tensor([133.1176], grad_fn=<SqueezeBackward1>)
MAPE loss: 0.9927150011062622
logits: tensor([302.6028], grad_fn=<SqueezeBackward1>)
MAPE loss: 0.9834396243095398
logits: tensor([608.6807], grad_fn=<SqueezeBackward1>)
MAPE loss: 0.9666891694068909
logits: tensor([1065.5330], grad_fn=<SqueezeBackward1>)
MAPE loss: 0.941687285900116
logits: tensor([1587.5902], grad_fn=<SqueezeBackward1>)
MAPE loss: 0.9131170511245728
logits: tensor([2552.9136], grad_fn=<SqueezeBackward1>)
MAPE loss: 0.8602883815765381
logits: tensor([3773.1409], grad_fn=<SqueezeBackward1>)
MAPE loss: 0.7935099005699158
logits: tensor([4501.8862], grad_fn=<SqueezeBackward1>)
MAPE loss: 0.7536283135414124
logits: tensor([7636.7534], grad_fn=<SqueezeBackward1>)
MAPE loss: 0.5820685625076294
logits: tensor([9148.1602], grad_fn=<SqueezeBackward1>)
MAPE loss: 0.49935483932495117
logits: ten

### Loss Curve Plot

In [322]:
px.line(train_losses)

In [323]:
px.line(val_losses)

### Random test sample

In [324]:
preds = []
ground_truth = []
vehicle_counts = []


for i in range(20):
    random_idx = np.random.randint(0,len(val_data))
    x, y = val_data[random_idx]
    vehicle_counts.append(x[0])
    ground_truth.append(float(y))
    pred_y = float(nn_model(x)[0])
    preds.append(pred_y)

df = pd.DataFrame({'vehicle_count': vehicle_counts, 'ground_truth': ground_truth, 'predictions': preds})

px.scatter(df, x='vehicle_count', y=['ground_truth', 'predictions'])

## Save Model

In [325]:
torch.save(nn_model.state_dict(), NN_MODEL_PATH)